In [2]:
import pandas as pd
fifa23=pd.read_csv('../data/male_players.csv')

- Removing rows where 'player_id' is null and modify the original dataframe

In [3]:
fifa23.dropna(subset=['player_id'], inplace=True)

In [4]:
fifa23.info()

<class 'pandas.core.frame.DataFrame'>
Index: 524287 entries, 0 to 1048545
Columns: 110 entries, player_id to player_face_url
dtypes: float64(63), object(47)
memory usage: 444.0+ MB


In [5]:
fifa23['fifa_version'].value_counts()

fifa_version
22.0    357613
23.0    166674
Name: count, dtype: int64

In [6]:
fifa22=fifa23[fifa23['fifa_version'] == 22]

In [7]:
fifa22

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
333321,158023.0,/player/158023/lionel-messi/220064,22.0,64.0,7/18/2022,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",92.0,92.0,...,64+3,64+3,65+3,60+3,50+3,50+3,50+3,60+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png
333323,188545.0,/player/188545/robert-lewandowski/220064,22.0,64.0,7/18/2022,R. Lewandowski,Robert Lewandowski,ST,92.0,92.0,...,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png
333325,20801.0,/player/20801/c-ronaldo-dos-santos-aveiro/220064,22.0,64.0,7/18/2022,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,ST,91.0,91.0,...,59+3,59+3,63+3,59+3,53+3,53+3,53+3,59+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png
333327,165153.0,/player/165153/karim-benzema/220064,22.0,64.0,7/18/2022,K. Benzema,Karim Benzema,"CF, ST",91.0,91.0,...,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3,https://cdn.sofifa.net/players/165/153/22_120.png
333329,192985.0,/player/192985/kevin-de-bruyne/220064,22.0,64.0,7/18/2022,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91.0,91.0,...,79+3,79+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048537,192708.0,/player/192708/carlos-eduardo-de-oliveira-alve...,22.0,45.0,4/21/2022,Carlos Eduardo,Carlos Eduardo de Oliveira Alves,"CAM, LM",76.0,76.0,...,68+2,68+2,67+2,66+2,65+2,65+2,65+2,66+2,15+2,https://cdn.sofifa.net/players/192/708/22_120.png
1048539,193116.0,/player/193116/maxime-gonalons/220045,22.0,45.0,4/21/2022,M. Gonalons,Maxime Gonalons,"CDM, CM",76.0,76.0,...,75+1,75+1,69+3,70+3,75+1,75+1,75+1,70+3,16+3,https://cdn.sofifa.net/players/193/116/22_120.png
1048541,193133.0,/player/193133/jens-stryger-larsen/220045,22.0,45.0,4/21/2022,J. Larsen,Jens Stryger Larsen,"LWB, RWB",76.0,76.0,...,73+2,73+2,74+2,73+2,71+2,71+2,71+2,73+2,15+2,https://cdn.sofifa.net/players/193/133/22_120.png
1048543,195668.0,/player/195668/joel-robles-blazquez/220045,22.0,45.0,4/21/2022,Joel Robles,Joel Robles Blázquez,GK,76.0,76.0,...,25+2,25+2,25+2,25+2,26+2,26+2,26+2,25+2,75+1,https://cdn.sofifa.net/players/195/668/22_120.png
